# ## IMPORTS

In [51]:
import cv2  # Import OpenCV for image processing
import mediapipe as mp  # Import MediaPipe for hand tracking
import pyautogui  # Import PyAutoGUI for GUI automation
import random  # Import random for generating random numbers
import util  # Import custom utility functions
from pynput.mouse import Button, Controller  # Import mouse control from pynput

# ## Mouse Controller

In [52]:
# Create a mouse controller instance
mouse = Controller()

# ## Screen Dimensions

In [53]:
# Get the screen width and height
screen_width, screen_height = pyautogui.size()

# ## MediaPipe Hands Initialization

In [54]:
# Initialize MediaPipe Hands module
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,  # Set to False for real-time detection
    model_complexity=1,  # Set model complexity (0 or 1)
    min_detection_confidence=0.7,  # Minimum confidence for detection
    min_tracking_confidence=0.7,  # Minimum confidence for tracking
    max_num_hands=1  # Maximum number of hands to track
)

# ## Function Definitions

In [55]:
# ### Find Finger Tip
# Function to find the tip of the index finger
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:  # Check if hand landmarks are detected
        hand_landmarks = processed.multi_hand_landmarks[0]  # Get the first hand's landmarks
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]  # Return index finger tip landmark
    return None  # Return None if no hand is detected

In [56]:
# ### Move Mouse
# Function to move the mouse to the position of the index finger tip
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:  # Check if the finger tip is detected
        x = int(index_finger_tip.x * screen_width)  # Calculate x position
        y = int(index_finger_tip.y * screen_height)  # Calculate y position
        pyautogui.moveTo(x, y)  # Move the mouse to the calculated position

In [57]:
# ### Gesture Detection Functions
# Function to determine if a left click gesture is made
def is_left_click(landmarks_list, thumb_index_dist):
    return (util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and
            util.get_distance([landmarks_list[9], landmarks_list[10], landmarks_list[12]]) > 90 and
            util.get_angle(landmarks_list[13], landmarks_list[14], landmarks_list[16]) > 90 and  # Ring finger up
            thumb_index_dist > 50)

# Function to determine if a right click gesture is made
def is_right_click(landmarks_list, thumb_index_dist):
    return (util.get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 50 and
            util.get_distance([landmarks_list[5], landmarks_list[6], landmarks_list[8]]) > 90 and
            thumb_index_dist > 50)

# Function to determine if a double click gesture is made
def is_double_click(landmarks_list, thumb_index_dist):
    return (util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and
            util.get_distance([landmarks_list[9], landmarks_list[10], landmarks_list[12]]) < 50 and
            thumb_index_dist > 50)

# Function to determine if a screenshot gesture is made
def is_screenshot(landmarks_list, thumb_index_dist):
    return (util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and
            util.get_distance([landmarks_list[9], landmarks_list[10], landmarks_list[12]]) < 50 and
            thumb_index_dist < 50)

# ### Detect Gestures

In [58]:
# Function to detect gestures based on hand landmarks
def detect_gestures(frame, landmarks_list, processed):
    if len(landmarks_list) >= 21:  # Check if enough landmarks are detected
        index_finger_tip = find_finger_tip(processed)  # Find the index finger tip
        thumb_index_distance = util.get_distance([landmarks_list[4], landmarks_list[5]])  # Calculate distance between thumb and index finger

        # Move mouse if thumb and index finger are close and index finger is raised
        if thumb_index_distance < 50 and util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) > 90:
            move_mouse(index_finger_tip)

        # LEFT CLICK
        elif is_left_click(landmarks_list, thumb_index_distance):
            mouse.press(Button.left)  # Press left mouse button
            mouse.release(Button.left)  # Release left mouse button
            cv2.putText(frame, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  # Display text on frame

        # RIGHT CLICK
        elif is_right_click(landmarks_list, thumb_index_distance):
            mouse.press(Button.right)  # Press right mouse button
            mouse.release(Button.right)  # Release right mouse button
            cv2.putText(frame, "Right Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)  # Display text on frame

        # DOUBLE CLICK
        elif is_double_click(landmarks_list, thumb_index_distance):
            pyautogui.doubleClick()  # Perform double click
            cv2.putText(frame, "Double Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)  # Display text on frame

        # SCREENSHOT
        elif is_screenshot(landmarks_list, thumb_index_distance):
            iml = pyautogui.screenshot()  # Take a screenshot
            label = random.randint(1, 1000)  # Generate a random label for the screenshot
            iml.save(f'my_screenshot_{label}.png')  # Save the screenshot with the label
            cv2.putText(frame, "Screenshot Taken", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)  # Display text on frame

# ## Main Function

In [59]:
# Main function to run the application
def main():
    cap = cv2.VideoCapture(0)  # Start video capture from the webcam
    draw = mp.solutions.drawing_utils  # Initialize drawing utilities for MediaPipe

    try:
        while cap.isOpened():  # Loop until the webcam is opened
            ret, frame = cap.read()  # Read a frame from the webcam

            if not ret:  # Break the loop if no frame is captured
                break
            frame = cv2.flip(frame, 1)  # Flip the frame horizontally
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB
            processed = hands.process(frameRGB)  # Process the frame for hand landmarks

            landmarks_list = []  # Initialize list to store landmarks

            if processed.multi_hand_landmarks:  # Check if hand landmarks are detected
                hand_landmarks = processed.multi_hand_landmarks[0]  # Get the first hand's landmarks
                draw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)  # Draw landmarks on the frame

                for lm in hand_landmarks.landmark:  # Loop through each landmark
                    landmarks_list.append((lm.x, lm.y))  # Append landmark coordinates to the list

            detect_gestures(frame, landmarks_list, processed)  # Detect gestures based on landmarks

            cv2.imshow('Frame', frame)  # Display the frame with landmarks
            if cv2.waitKey(1) & 0xFF == ord('q'):  # Exit if 'q' is pressed
                break

    finally:
        cap.release()  # Release the webcam
        cv2.destroyAllWindows()  # Close all OpenCV windows

# ## Entry Point

In [60]:
# Entry point of the program
if __name__ == "__main__":
    main()  # Call the main function to run the application